In [1]:
#!pip install pandas
#%matplotlib inline
import math
import numpy as np
import pandas as pd
import pandasql as ps
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import IPython.display as ipd

import ast

import utils

from collections import defaultdict
from scipy.stats.stats import pearsonr

plt.rcParams['axes.formatter.useoffset']=False
pd.set_option('display.max_columns', None)

In [2]:
echonest = pd.read_csv('fma_metadata_progetto/echonest.csv', index_col=0, header=[0, 1, 2])
features = pd.read_csv('fma_metadata_progetto/features.csv', index_col=0, header=[0, 1, 2])
genres = pd.read_csv('fma_metadata_progetto/genres.csv', index_col=0)
tracks = pd.read_csv('fma_metadata_progetto/tracks.csv', index_col=0, header=[0, 1])

#raw_albums = pd.read_csv('fma_metadata_progetto/raw_albums.csv')
#raw_artists = pd.read_csv('fma_metadata_progetto/raw_artists.csv')
#raw_echonest = pd.read_csv('fma_metadata_progetto/raw_echonest.csv')
#raw_generes = pd.read_csv('fma_metadata_progetto/raw_genres.csv')
#raw_tracks = pd.read_csv('fma_metadata_progetto/raw_tracks.csv')


COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
for column in COLUMNS:
    tracks[column] = tracks[column].map(ast.literal_eval)

COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
           ('album', 'date_created'), ('album', 'date_released'),
           ('artist', 'date_created'), ('artist', 'active_year_begin'),
           ('artist', 'active_year_end')]
for column in COLUMNS:
    tracks[column] = pd.to_datetime(tracks[column])

SUBSETS = ('small', 'medium', 'large')
try:
    cat_type = CategoricalDtype(categories=SUBSETS, ordered=True)
    tracks['set', 'subset'] = tracks['set', 'subset'].astype(cat_type)
except ValueError:
    # the categories and ordered arguments were removed in pandas 0.25
    tracks['set', 'subset'] = tracks['set', 'subset'].astype(
             pd.CategoricalDtype(categories=SUBSETS, ordered=True))

COLUMNS = [('track', 'genre_top'), ('track', 'license'),
           ('album', 'type'), ('album', 'information'),
           ('artist', 'bio')]
for column in COLUMNS:
    tracks[column] = tracks[column].astype('category')

# Inizio

## Task

Vogliamo prevedere/predirre il genere.
Usiamo solo tracks e features (per ora).

Non possiamo usare echonest perche contiene i dati di solo 13k canzoni (c.a 12%).

proviamo a fare un modello usando features e uno non usandole, prediamo ed affiniamo il migliore dei due.

### Selezione attributi

In [3]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   (album, comments)            106574 non-null  int64         
 1   (album, date_created)        103045 non-null  datetime64[ns]
 2   (album, date_released)       70294 non-null   datetime64[ns]
 3   (album, engineer)            15295 non-null   object        
 4   (album, favorites)           106574 non-null  int64         
 5   (album, id)                  106574 non-null  int64         
 6   (album, information)         83149 non-null   category      
 7   (album, listens)             106574 non-null  int64         
 8   (album, producer)            18060 non-null   object        
 9   (album, tags)                106574 non-null  object        
 10  (album, title)               105549 non-null  object        
 11  (album, tracks)           

Da tracks.album
- date_released --> solo l'anno
- favorites
- type
- tags
- tracks
- listens
- title

Da tracks.set
- subset

Da tracks.artist
- favorites
- nazione(location)
- tags
- name

Da tracks.track
- duration
- favorites
- genre_top
- genres_all
- genres
- interest
- listens
- license
- tags
- title
- number
- bit_rate

Da features: TUTTO

In [3]:
flat_df = pd.DataFrame()

# Da album
flat_df["ALBUM_date_released"] = tracks.album.date_released
flat_df["ALBUM_date_released_year"] = tracks.album.date_released.map(lambda x: str(x)[0:4])
flat_df["ALBUM_favorites"] = tracks.album.favorites
flat_df["ALBUM_type"] = tracks.album.type
flat_df["ALBUM_tags"] = tracks.album.tags
flat_df["ALBUM_tracks"] = tracks.album.tracks
flat_df["ALBUM_listens"] = tracks.album.listens
flat_df["ALBUM_title"] = tracks.album.title


#Da set
flat_df["SET_subset"] = tracks.set.subset


#Da artist
flat_df["ARTIST_favorites"] = tracks.artist.favorites
flat_df["ARTIST_location"] = tracks.artist.location
flat_df["ARTIST_tags"] = tracks.artist.tags
flat_df["ARTIST_name"] = tracks.artist.name


#da track
flat_df["TRACK_duration"] = tracks.track.duration
flat_df["TRACK_favorites"] = tracks.track.favorites
flat_df["TRACK_genre_top"] = tracks.track.genre_top
flat_df["TRACK_genres_all"] = tracks.track.genres_all
flat_df["TRACK_genres"] = tracks.track.genres
flat_df["TRACK_interest"] = tracks.track.interest
flat_df["TRACK_listens"] = tracks.track.listens
flat_df["TRACK_license"] = tracks.track.license
flat_df["TRACK_tags"] = tracks.track.tags
flat_df["TRACK_title"] = tracks.track.title
flat_df["TRACK_bitrate"] = tracks.track.bit_rate
flat_df["TRACK_number"] = tracks.track.number

flat_df_no_features = flat_df.copy()


#da features
for l1 in features.columns:
    name = ""
    for l2 in l1:
        name = name + l2
        
    flat_df[name] = features[l1]


flat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Columns: 543 entries, ALBUM_date_released to zcrstd01
dtypes: category(4), datetime64[ns](1), float64(518), int64(10), object(10)
memory usage: 439.5+ MB


In [19]:
flat_df.to_csv("flat_df.csv")
flat_df_no_features.to_csv("flat_df_no_features.csv")

In [20]:
flat_df.head()

,ALBUM_date_released,ALBUM_date_released_year,ALBUM_favorites,ALBUM_type,ALBUM_tags,ALBUM_tracks,ALBUM_listens,ALBUM_title,SET_subset,ARTIST_favorites,ARTIST_location,ARTIST_tags,ARTIST_name,TRACK_duration,TRACK_favorites,TRACK_genre_top,TRACK_genres_all,TRACK_genres,TRACK_interest,TRACK_listens,TRACK_license,TRACK_tags,TRACK_title,TRACK_bitrate,TRACK_number,chroma_censkurtosis01,chroma_censkurtosis02,chroma_censkurtosis03,chroma_censkurtosis04,chroma_censkurtosis05,chroma_censkurtosis06,chroma_censkurtosis07,chroma_censkurtosis08,chroma_censkurtosis09,chroma_censkurtosis10,chroma_censkurtosis11,chroma_censkurtosis12,chroma_censmax01,chroma_censmax02,chroma_censmax03,chroma_censmax04,chroma_censmax05,chroma_censmax06,chroma_censmax07,chroma_censmax08,chroma_censmax09,chroma_censmax10,chroma_censmax11,chroma_censmax12,chroma_censmean01,chroma_censmean02,chroma_censmean03,chroma_censmean04,chroma_censmean05,chroma_censmean06,chroma_censmean07,chroma_censmean08,chroma_censmean09,chroma_censmean10,chroma_censmean11,chroma_censmean12,chroma_censmedian01,chroma_censmedian02,chroma_censmedian03,chroma_censmedian04,chroma_censmedian05,chroma_censmedian06,chroma_censmedian07,chroma_censmedian08,chroma_censmedian09,chroma_censmedian10,chroma_censmedian11,chroma_censmedian12,chroma_censmin01,chroma_censmin02,chroma_censmin03,chroma_censmin04,chroma_censmin05,chroma_censmin06,chroma_censmin07,chroma_censmin08,chroma_censmin09,chroma_censmin10,chroma_censmin11,chroma_censmin12,chroma_censskew01,chroma_censskew02,chroma_censskew03,chroma_censskew04,chroma_censskew05,chroma_censskew06,chroma_censskew07,chroma_censskew08,chroma_censskew09,chroma_censskew10,chroma_censskew11,chroma_censskew12,chroma_censstd01,chroma_censstd02,chroma_censstd03,chroma_censstd04,chroma_censstd05,chroma_censstd06,chroma_censstd07,chroma_censstd08,chroma_censstd09,chroma_censstd10,chroma_censstd11,chroma_censstd12,chroma_cqtkurtosis01,chroma_cqtkurtosis02,chroma_cqtkurtosis03,chroma_cqtkurtosis04,chroma_cqtkurtosis05,chroma_cqtkurtosis06,chroma_cqtkurtosis07,chroma_cqtkurtosis08,chroma_cqtkurtosis09,chroma_cqtkurtosis10,chroma_cqtkurtosis11,chroma_cqtkurtosis12,chroma_cqtmax01,chroma_cqtmax02,chroma_cqtmax03,chroma_cqtmax04,chroma_cqtmax05,chroma_cqtmax06,chroma_cqtmax07,chroma_cqtmax08,chroma_cqtmax09,chroma_cqtmax10,chroma_cqtmax11,chroma_cqtmax12,chroma_cqtmean01,chroma_cqtmean02,chroma_cqtmean03,chroma_cqtmean04,chroma_cqtmean05,chroma_cqtmean06,chroma_cqtmean07,chroma_cqtmean08,chroma_cqtmean09,chroma_cqtmean10,chroma_cqtmean11,chroma_cqtmean12,chroma_cqtmedian01,chroma_cqtmedian02,chroma_cqtmedian03,chroma_cqtmedian04,chroma_cqtmedian05,chroma_cqtmedian06,chroma_cqtmedian07,chroma_cqtmedian08,chroma_cqtmedian09,chroma_cqtmedian10,chroma_cqtmedian11,chroma_cqtmedian12,chroma_cqtmin01,chroma_cqtmin02,chroma_cqtmin03,chroma_cqtmin04,chroma_cqtmin05,chroma_cqtmin06,chroma_cqtmin07,chroma_cqtmin08,chroma_cqtmin09,chroma_cqtmin10,chroma_cqtmin11,chroma_cqtmin12,chroma_cqtskew01,chroma_cqtskew02,chroma_cqtskew03,chroma_cqtskew04,chroma_cqtskew05,chroma_cqtskew06,chroma_cqtskew07,chroma_cqtskew08,chroma_cqtskew09,chroma_cqtskew10,chroma_cqtskew11,chroma_cqtskew12,chroma_cqtstd01,chroma_cqtstd02,chroma_cqtstd03,chroma_cqtstd04,chroma_cqtstd05,chroma_cqtstd06,chroma_cqtstd07,chroma_cqtstd08,chroma_cqtstd09,chroma_cqtstd10,chroma_cqtstd11,chroma_cqtstd12,chroma_stftkurtosis01,chroma_stftkurtosis02,chroma_stftkurtosis03,chroma_stftkurtosis04,chroma_stftkurtosis05,chroma_stftkurtosis06,chroma_stftkurtosis07,chroma_stftkurtosis08,chroma_stftkurtosis09,chroma_stftkurtosis10,chroma_stftkurtosis11,chroma_stftkurtosis12,chroma_stftmax01,chroma_stftmax02,chroma_stftmax03,chroma_stftmax04,chroma_stftmax05,chroma_stftmax06,chroma_stftmax07,chroma_stftmax08,chroma_stftmax09,chroma_stftmax10,chroma_stftmax11,chroma_stftmax12,chroma_stftmean01,chroma_stftmean02,chroma_stftmean03,chroma_stftmean04,chroma_stftmean05,chroma_stftmean06,chroma_stftmean07,chroma_stftmean08,chroma_stftmean

In [13]:
flat_df_no_features[flat_df_no_features.SET_subset == "medium"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17000 entries, 3 to 155314
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ALBUM_date_released       11755 non-null  datetime64[ns]
 1   ALBUM_date_released_year  17000 non-null  object        
 2   ALBUM_favorites           17000 non-null  int64         
 3   ALBUM_type                16568 non-null  category      
 4   ALBUM_tags                17000 non-null  object        
 5   ALBUM_tracks              17000 non-null  int64         
 6   ALBUM_listens             17000 non-null  int64         
 7   SET_subset                17000 non-null  category      
 8   ARTIST_favorites          17000 non-null  int64         
 9   ARTIST_location           11510 non-null  object        
 10  ARTIST_tags               17000 non-null  object        
 11  TRACK_duration            17000 non-null  int64         
 12  TRACK_favorites  

In [15]:
tracks.track.number.unique()

array([  3,   4,   6,   1,   5,   8,   9,   0,   2,   7,  11,  10,  12,
        13,  14,  15,  16,  17,  19,  18,  20,  23,  21,  22,  25,  24,
       100,  66,  41,  42,  32,  40,  36,  30,  29,  28,  31,  39,  33,
        26,  27,  37,  35,  34,  43,  38,  44,  45,  54,  47,  50,  46,
        48,  49,  51,  52,  53,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  87, 255, 101, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217,  81,  82,  83,  84,  85,  86,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 121, 122, 123, 124,
       125, 126, 127, 128, 129, 134, 130, 131, 135, 136, 132, 133, 227,
       138, 142, 137, 145, 153, 147, 144, 148, 151, 149, 146, 150, 139,
       140, 152, 141, 143, 156, 160, 158, 154, 159, 155, 157, 16